## Globale Temperaturänderungen und Gefahren

### Teil 1
- Projektinformationen & Vorabrecherche
- Temperatur- & Geodaten einlesen, analysieren und aufbereiten

## 1. Projektziele

**Daten, aus zwei verschiedenen Quellen, mit ortsbezogenen Informationen zum Klimawandel,\
werden deskriptiv analysiert, aufbereitet und visualisiert.**

### ETL-Prozess
- Die Verarbeitungsschritte sollen in einer möglichst Arbeitsressourcen schonenden Weise stattfinden,\
  u.a. um größere Datenmengen auch auf leistungsschwachen Systemen verarbeiten zu können.
- Daten werden zu Teil umstrukturiert, durch Auslagerung in separate Frames
- Redundanzen werden verringert
- Für die Betrachtung unnötige Daten werden gekürzt, zusammengefasst, entfernt
- Speicherverwendung wird berücksichtigt
- Die Zusammenführung der Daten findet auch unter dem Aspekt des Datenerhaltes statt

### Analyse & Visualisierung
- Daten werden vorweg eingelesen, gesichtet und geprüft, um die Verarbeitungsschritte möglichst zielorientiert zu gestalten.
- Während des Verarbeitungsprozesses wird kontinuierlich geprüft, ob eine Anpassung der Verarbeitungslogik erforderlich ist. 
- Es kommen u.a. NumPy, Pandas, ... zum Einsatz.
- Zur Visualisierung wird Ploty Express verwendet.

### Speicherbedarf
Übersicht der Speicherverwendung verschiedener Datentypen im Vergleich

---

#### String

| Framework | String-Typ       | Speicherbedarf (Formel)     | Bemerkung                                                                  |
|-----------|------------------|-----------------------------|----------------------------------------------------------------------------|
| Python    | `str`            | 49 Bytes + 1 Byte/Char      | Hoher Objekt-Overhead, flexibel, ineffizient für große Datenmengen.        |
| NumPy     | `np.str_`        | 4 Bytes/Char (UTF-32)       | Fester Puffer, ineffizient für kurze Strings.                              |
| Pandas    | `object`-Dtype   | 57 Bytes + 1 Byte/Char      | Zeiger (8 Bytes) + Python-String.                                          |
| Pandas    | `string` (Arrow) | **4 Bytes + 1 Byte/Char**   | Effizienter Speicher mit Apache Arrow-Backend.                             |
| PySpark   | `StringType`     | ca. 40 Bytes + 2 Bytes/Char | Serialisierung über JVM/Arrow, optimiert für verteilte Big-Data-Szenarien. |

---

#### Float

| Framework | Float-Typ    | Speicherbedarf | Präzision (Beispiel)                         | Anwendungsszenario                                                           |
|-----------|--------------|----------------|----------------------------------------------|------------------------------------------------------------------------------|
| Python    | `float`      | 24 Bytes       | ≈15–17 Dezimalstellen                        | Allgemeine Berechnungen, kleine Datensätze.                                  |
| NumPy     | `float16`    | 2 Bytes        | ≈3 Dezimalstellen                            | Machine Learning (z.B. TensorFlow-Modelle), Speichereffizienz vor Präzision. |
| NumPy     | `float32`    | 4 Bytes        | ≈6–7 Dezimalstellen (≈1,7 m bei Koordinaten) | Geodaten (Heatmaps, Cluster), Echtzeit-Sensordaten.                          |
| NumPy     | `float64`    | 8 Bytes        | ≈15–17 Dezimalstellen (≈3,16 nm)             | Wissenschaftliche Berechnungen, Vermessung, millimetergenaue Anwendungen.    |
| Pandas    | `float32`    | 4 Bytes        | Wie NumPy                                    | Reduktion des Speicherbedarfs bei großen DataFrames.                         |
| Pandas    | `float64`    | 8 Bytes        | Wie NumPy                                    | Standard für präzise numerische Operationen.                                 |
| PySpark   | `FloatType`  | 4 Bytes        | Wie `float32`                                | Big-Data-Analysen, Speichereffizienz in verteilten Systemen.                 |
| PySpark   | `DoubleType` | 8 Bytes        | Wie `float64`                                | Hochpräzise Berechnungen in Spark-Anwendungen.                               |

---

#### Date

| Framework | Date-Typ         | Speicherbedarf | Auflösung    | Anwendungsszenario                                                |
|-----------|------------------|----------------|--------------|-------------------------------------------------------------------|
| Python    | `datetime.date`  | ≥48 Bytes      | Tage         | Kleine Datensätze, einfache Datumsoperationen.                    |
| NumPy     | `datetime64[D]`  | 8 Bytes        | Tage         | Effiziente Speicherung von Datumsreihen (z.B. Zeitreihenanalyse). |
| Pandas    | `datetime64[ns]` | 8 Bytes        | Nanosekunden | Standard für Zeitstempel in Pandas, hohe Präzision.               |
| PySpark   | `DateType`       | 4 Bytes        | Tage         | Optimiert für verteilte Verarbeitung in Spark-Clustern.           |

---

#### Empfehlungen
- **Strings:** Pandas mit Arrow-Backend (`dtype="string"`) oder PySpark für Big Data.
- **Floats:** `float32` für Geodaten (wenn ≈2 m Toleranz akzeptabel) und `float64` für Präzision.
- **Datum:** Pandas `datetime64[ns]` für maximale Flexibilität, PySpark `DateType` für verteilte Systeme.

*<p style="text-align: right;">Antwort von Perplexity (modifiziert)</p>*

---

## 2. Datenquellen

#### Climate Change
Climate Change: Earth Surface Temperature Data\
Monatliche Durchschnittstemperaturen zu Städten von November 1743 bis September 2013

> https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data

> `GlobalLandTemperaturesByCity.csv` - ca. 520 MB

#### Climate Hazards
2023 Cities Climate Hazards\
Bisherige sowie prognostizierte Auswirkungen des Klimawandels in Städten - Stand 2023

> https://data.cdp.net/Climate-Hazards/2023-Cities-Climate-Hazards/rng4-m4ks/about_data

> `2023_Cities_Climate_Hazards_20250506.csv` - ca. 6 MB

<br>

---

## 3. Arbeitsumgebung
- Windows 10 Education 22H2 (per RDP)
- Python 3.12.9
- JupyterLab Desktop 4.2.5-1

<br>

---

In [20]:
import numpy as np
import pandas as pd
from lat_lon_parser import parse
from openlocationcode import openlocationcode as olc

pd.set_option('display.max_colwidth', None)  # Spaltenbreitenberschränkung aufheben

# Temperatur- & Geodaten einlesen und aufbereiten

## 1. Einlesen

### Temperaturdaten [ celsius_df ]
Ausgewählte Spalten werden aus GlobalLandTemperaturesByCity.csv ins das Dataframe `celsius_df` geladen:
| Spalte | Beschreibung | Anmerkung |
|-|-|-|
| dt | Monat des Temperaturwerts | Datum im String 'yyyy-mm-dd' |
| City | Stadt | bis zu 3 Geo-Koordinaten | unterschiedliche Orte, selber Name
| AverageTemperature | AVG Temperatur pro Monat | Temperaturwerte unvollständig |
| Latitude | Breitengrade | Dezimalgrad mit Richtungsbuchstaben
| Longitude | Längengrade | Dezimalgrad mit Richtungsbuchstaben

### Geodaten [ geo_df ]
Ausgewählte Spalten werden aus GlobalLandTemperaturesByCity.csv ins das Dataframe `geo_df` geladen:
| Spalte | Beschreibung | Anmerkung |
|-|-|-|
| City | Stadt | bis zu 3 Geo-Koordinaten | unterschiedliche Orte, selber Name
| Country | Land |
| Latitude | Breitengrade | Dezimalgrad mit Richtungsbuchstaben
| Longitude | Längengrade | Dezimalgrad mit Richtungsbuchstaben

---

### pd.info()

Pandas zeigt die "Non-Null Count"-Spalte nur dann automatisch an, wenn der DataFrame eine bestimmte Größe unterschreitet.
- Einstellungen: pandas.options.display.max_info_rows und max_info_columns (default: 1.690.785 Zeilen)
- Der Parameter `show_counts=True` erzwingt die "Non-Null Count"-Ausgabe

Der Speicherbedarf wird nur anhand des Datentyps und der Anzahl der Zeilen geschätzt.
- Für numerische Typen ist das exakt, weil jeder Wert im Speicher gleich groß ist
- Für object-Spalten wird nur der Zeiger-Speicher gezählt, nicht der tatsächliche Inhalt
- Bei vielen langen oder sehr vielen unterschiedlichen Strings kann der Unterschied um ein Vielfaches höher sein,\
  weil jeder String als eigenes Python-Objekt im Speicher liegt und zusätzlichen Overhead verursacht
- Der Parameter `memory_usage='deep'` ermittelt die tatsächliche Speicherbelegung der object-Spalten im RAM (kostet mehr Rechenzeit)


In [21]:
# Einlesen von Datum, Stadt und Temperatur aus der Haupt-Datenquelle --> ca. 5 Sekunden
celsius_df = pd.read_csv('GlobalLandTemperaturesByCity.csv',
                 usecols=['dt', 'City', 'AverageTemperature', 'Latitude', 'Longitude'],
                 dtype={'AverageTemperature': 'float32',
                        'City': 'string'
                       }
                )

#### celsius_df.info()

Der DataFrame enthält **8.599.212 Zeilen**, weswegen "Non-Null Count" nicht ausgegeben wird.\
Es wird 295.2+ MB geschätzt und **1.8 GB** ermittelt.

In [22]:
celsius_df.info()
print('')
celsius_df.info(show_counts=True, memory_usage='deep')  # ca. 4 Sekunden

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 5 columns):
 #   Column              Dtype  
---  ------              -----  
 0   dt                  object 
 1   AverageTemperature  float32
 2   City                string 
 3   Latitude            object 
 4   Longitude           object 
dtypes: float32(1), object(3), string(1)
memory usage: 295.2+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 5 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   dt                  8599212 non-null  object 
 1   AverageTemperature  8235082 non-null  float32
 2   City                8599212 non-null  string 
 3   Latitude            8599212 non-null  object 
 4   Longitude           8599212 non-null  object 
dtypes: float32(1), object(3), string(1)
memory usage: 1.8 GB


In [23]:
# Beispielauszug
celsius_df.sample(5)

,dt,AverageTemperature,City,Latitude,Longitude
854890,1976-02-01,10.273,Berkeley,37.78N,122.03W
1124094,1759-03-01,4.856,Brescia,45.81N,10.38E
3644108,1888-09-01,27.909,Karachi,24.92N,67.39E
2065351,1924-05-01,13.968,Dortmund,52.24N,7.88E
7050299,1783-11-01,NaN,Siverek,37.78N,38.64E


In [24]:
# Einlesen von Stadt, Land und Geo-Koordinaten aus der Haupt-Datenquelle --> ca. 6 Sekunden
geo_df = pd.read_csv('GlobalLandTemperaturesByCity.csv',
                 usecols=['City', 'Country', 'Latitude', 'Longitude'],
                 dtype={'City': 'string',
                        'Country': 'string'
                       }
                )

#### geo_df.info()

Der DataFrame enthält **8.599.212 Zeilen**, weswegen "Non-Null Count" nicht ausgegeben wird.\
Es wird 262.4+ MB geschätzt und **1.8 GB** ermittelt.

In [25]:
geo_df.info()
print('')
geo_df.info(show_counts=True, memory_usage='deep')  # ca. 6 Sekunden

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   City       string
 1   Country    string
 2   Latitude   object
 3   Longitude  object
dtypes: object(2), string(2)
memory usage: 262.4+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   City       8599212 non-null  string
 1   Country    8599212 non-null  string
 2   Latitude   8599212 non-null  object
 3   Longitude  8599212 non-null  object
dtypes: object(2), string(2)
memory usage: 1.8 GB


In [26]:
# Beispielauszug
geo_df.sample(5)

,City,Country,Latitude,Longitude
6590214,Salto,Uruguay,31.35S,58.43W
1525346,Chernivtsi,Ukraine,49.03N,26.94E
5075731,Namangan,Uzbekistan,40.99N,72.43E
1258244,Cabanatuan,Philippines,15.27N,120.83E
410720,Armenia,Colombia,4.02N,76.34W


<br>

## 2. Bereinigung & Transformation

### 2.1. Zeilen ohne Temperaturwerte löschen

364.130 Zeilen in `celsius_df` enthalten keine Temperaturwerte und werden gelöscht
- Zeilen vorher: 8.599.212
- Zeilen nacher: 8.235.082

Im Anschluss sind alle Felder "non-null"

In [27]:
# Temperaturwerte sind unvollständig / Beispiel Berlin nach Datum asc
print(celsius_df[celsius_df['City'].isin(['Berlin'])].sort_values(by='dt', ascending=True).head(10).to_string(index=False))

        dt  AverageTemperature   City Latitude Longitude
1743-11-01               6.326 Berlin   52.24N    13.14E
1743-12-01                 NaN Berlin   52.24N    13.14E
1744-01-01                 NaN Berlin   52.24N    13.14E
1744-02-01                 NaN Berlin   52.24N    13.14E
1744-03-01                 NaN Berlin   52.24N    13.14E
1744-04-01               9.536 Berlin   52.24N    13.14E
1744-05-01              12.579 Berlin   52.24N    13.14E
1744-06-01              14.809 Berlin   52.24N    13.14E
1744-07-01              17.275 Berlin   52.24N    13.14E
1744-08-01                 NaN Berlin   52.24N    13.14E


In [28]:
# Zeilen ohne Temperaturwerte aus celsius_df entfernen und internen Index neu bilden (ohne Index-Sicherung)
celsius_df = celsius_df.dropna(subset=['AverageTemperature']).reset_index(drop=True)

# Fehlende Temperaturwerte entfernt / Beispiel Berlin nach Datum asc
print(celsius_df[celsius_df['City'].isin(['Berlin'])].sort_values(by='dt', ascending=True).head(6).to_string(index=False))
print('')

# Zeilen mit Null Werten gelöscht / Speicherbelegung um etwa 4 % reduziert 
celsius_df.info(show_counts=True, memory_usage='deep')

        dt  AverageTemperature   City Latitude Longitude
1743-11-01               6.326 Berlin   52.24N    13.14E
1744-04-01               9.536 Berlin   52.24N    13.14E
1744-05-01              12.579 Berlin   52.24N    13.14E
1744-06-01              14.809 Berlin   52.24N    13.14E
1744-07-01              17.275 Berlin   52.24N    13.14E
1744-09-01              14.119 Berlin   52.24N    13.14E

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8235082 entries, 0 to 8235081
Data columns (total 5 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   dt                  8235082 non-null  object 
 1   AverageTemperature  8235082 non-null  float32
 2   City                8235082 non-null  string 
 3   Latitude            8235082 non-null  object 
 4   Longitude           8235082 non-null  object 
dtypes: float32(1), object(3), string(1)
memory usage: 1.8 GB


### 2.2. Datum-String konvertieren
- Datum-String als Pandas `object` wird zu Pandas `datetime64[ns]` konvertiert
- Benötigt etwa **86 % weniger** Speicher beim vorliegenden Datumsformat "YYYY-MM-DD"

In [29]:
# Speicherbelegung davor
print(f'Speicherbelegung object: {int(round(celsius_df.dt.memory_usage(deep=True) / 1048576, 0))} MB')

# Datumskonvertierung
celsius_df['dt'] = pd.to_datetime(celsius_df['dt'])

# Speicherbelegung danach
print(f'Speicherbelegung datetime64[ns]: {int(round(celsius_df.dt.memory_usage(deep=True) / 1048576, 0))} MB')

Speicherbelegung object: 463 MB
Speicherbelegung datetime64[ns]: 63 MB


### 2.3. Geo-Koordinaten bereinigen und konvertieren
- Redundanzen aus der Mehrfachzuordnung von Geo-Koordinaten werden entfernt: `.drop_duplicates()`
- Koordinaten-Strings zu Float Konvertierung unter Verwendung des Moduls: *`lat_lon_parser.parse`*
  - Float benötigt weniger Speicher als String
  - Float wird zur Visualisierung von Geo-Koordinaten verwendet

In [30]:
""" Dauer etwa 50 Sekunden """

# Duplikate aus geo_df entfernen und Index neu bilden (ohne Index-Sicherung)
geo_df = geo_df.drop_duplicates().reset_index(drop=True)

# Geo-Spalten in geo_df konvertieren, mit parse-Funktion aus lat_lon_parser
geo_df['Latitude'] = geo_df['Latitude'].apply(parse).astype('float32')
geo_df['Longitude'] = geo_df['Longitude'].apply(parse).astype('float32')

# Geo-Spalten in celsius_df konvertieren, mit parse-Funktion aus lat_lon_parser --> ca. 50 Sekunden
celsius_df['Latitude'] = celsius_df['Latitude'].apply(parse).astype('float32')
celsius_df['Longitude'] = celsius_df['Longitude'].apply(parse).astype('float32')

# DataFrames Info
geo_df.info(memory_usage='deep')
print('')
celsius_df.info(show_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510 entries, 0 to 3509
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       3510 non-null   string 
 1   Country    3510 non-null   string 
 2   Latitude   3510 non-null   float32
 3   Longitude  3510 non-null   float32
dtypes: float32(2), string(2)
memory usage: 422.3 KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8235082 entries, 0 to 8235081
Data columns (total 5 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   dt                  8235082 non-null  datetime64[ns]
 1   AverageTemperature  8235082 non-null  float32       
 2   City                8235082 non-null  string        
 3   Latitude            8235082 non-null  float32       
 4   Longitude           8235082 non-null  float32       
dtypes: datetime64[ns](1), float32(3), string(1)
memory usage: 612.0 MB


### 2.4. City & Geo-Koordinaten als Schlüssel
- City kann bis zu 3 verschiedene Geo-Koordinaten haben
  - Es handelt sich hierbei um unterschiedliche Orte mit dem selben Namen
- Eine Geo-Koordinate kann verschiedene City-Zuordnungen haben
  - Die Koordinate liegt relativ zentral zwischen den gelisteten Städten,\
    stellt aber nicht exakt den geografischen Mittelpunkt dar
  - Die Genauigkeit ist für dieses Vorhaben ausreichend

In [31]:
# City kann bis zu 3 verschiedene Geo-Koordinaten haben / Beispiel Springfield
print(geo_df[geo_df['City'].isin(['Springfield'])].to_string(index=False))

       City       Country  Latitude  Longitude
Springfield United States 37.779999 -93.559998
Springfield United States 39.380001 -89.480003
Springfield United States 42.590000 -72.000000


In [32]:
# Eine Geo-Koordinate kann verschiedene City-Zuordnungen haben / Beispiel Indien, West Gujarat
print(geo_df
      .query('Latitude == 21.700001 and Longitude == 70.099998')
      .to_string(index=False)
)

     City Country  Latitude  Longitude
   Gondal   India 21.700001  70.099998
 Jamnagar   India 21.700001  70.099998
   Jetpur   India 21.700001  70.099998
 Junagadh   India 21.700001  70.099998
Porbandar   India 21.700001  70.099998
   Rajkot   India 21.700001  70.099998
  Veraval   India 21.700001  70.099998


**City und Koordinaten müssen in Kombination als Schlüsselspalte dienen, um celsius_df und geo_df in Relation zu bringen.**

#### Open Location Code (Plus Codes)
Der Algorithmus "Open Location Code" von Google wird verwendet, um Geo-Koordinaten in einen\
eindeutigen, kurzen alphanumerischen String (Plus Code) umzuwandeln (z.B. 87G8Q72V+G6).
- Plus Codes sind weltweit eindeutig
- Jede Koordinate ergibt einen eindeutigen, kompakten Hash als String
- Genauigkeit lässt sich mit dem Argument codeLength steuern
  - 10 Zeichen ≈ 13,5 × 13,5 Meter (Standard)
  - 8 Zeichen ≈ 135 × 135 Meter (ausreichend)

In [33]:
# Beispiel
plus_encode = olc.encode(42.101391, -72.590279, codeLength=10)
plus_decode = olc.decode(plus_encode)
print(f'10 Zeichen: {plus_encode} | {plus_decode.latitudeCenter} | {plus_decode.longitudeCenter}')
plus_encode = olc.encode(42.101391, -72.590279, codeLength=8)
plus_decode = olc.decode(plus_encode)
print(f' 8 Zeichen: {plus_encode}{' '*3}| {plus_decode.latitudeCenter}{' '*10}| {plus_decode.longitudeCenter}')

10 Zeichen: 87J94C25+HV | 42.10143749999999 | -72.5903125
 8 Zeichen: 87J94C25+   | 42.10125          | -72.59125


In [34]:
""" Dauer etwa 90 Sekunden """

def gen_plus_code(df, lat_col='Latitude', lon_col='Longitude', code_length=8, new_col='plus_code'):
    """ Fügt einem DataFrame eine neue Spalte mit Plus Codes (Open Location Code) hinzu,
        basierend auf Latitude und Longitude Spalten.
        Aufruf: df = gen_plus_code(df) """
    
    df[new_col] = df.apply(
        lambda row: olc.encode(row[lat_col], row[lon_col], codeLength=code_length), axis=1
    )
    return df

# Plus Code in celsius_df erzeugen --> ca. 90 Sekunden
celsius_df = gen_plus_code(celsius_df)

# Plus Code in geo_df erzeugen
geo_df = gen_plus_code(geo_df)

# Geo-Koordinaten aus celsius_df entfernen
celsius_df = celsius_df.drop(columns=['Latitude', 'Longitude'])


#### Kombinierter Schlüssel
Weil City und Koordinaten kombiniert werden müssen, wurde folgender Ansatz gewählt:

##### Zwei separate Schlüsselspalten
- Klar lesbar, entspricht SQL-Mehrspalten-Keys
- Pandas ist für diesen Fall optimiert
- Kein zusätzlicher Speicher für eine Hilfsspalte
- Kein zusätzlicher Schritt zum zusammenführen der Spalten

**Die Kombination aus `City` und `plus_code` ergibt jetzt einen eindeutigen Schlüssel, der zum Mergen verwendet werden kann.**

In [35]:
# [df1] City mit verschiedenen Geo-Koordinaten und plus_code / Beispiel Springfield
# [df2] Geo-Koordinate mit verschiedenen City-Zuordnungen und plus_code / Beispiel Indien, West Gujarat

print('[-- celsius_df --]')
df1 = celsius_df[['City', 'plus_code']][celsius_df['City'].isin(['Springfield'])]
df2 = celsius_df[celsius_df['plus_code'] == '7JHGP32X+'][['City', 'plus_code']].drop_duplicates()
print(pd.concat([df1, df2]).drop_duplicates().to_string(index=False))
print('')
print('[-- geo_df --]')
df1 = geo_df[['City', 'plus_code', 'Latitude', 'Longitude', 'Country']][geo_df['City'].isin(['Springfield'])]
df2 = geo_df[['City', 'plus_code', 'Latitude', 'Longitude', 'Country']].query('Latitude == 21.700001 and Longitude == 70.099998')
print(pd.concat([df1, df2]).drop_duplicates().to_string(index=False))

[-- celsius_df --]
       City plus_code
Springfield 8698QCHR+
Springfield 86FG9GJ9+
Springfield 87JCH2R2+
     Gondal 7JHGP32X+
   Jamnagar 7JHGP32X+
     Jetpur 7JHGP32X+
   Junagadh 7JHGP32X+
  Porbandar 7JHGP32X+
     Rajkot 7JHGP32X+
    Veraval 7JHGP32X+

[-- geo_df --]
       City plus_code  Latitude  Longitude       Country
Springfield 8698QCHR+ 37.779999 -93.559998 United States
Springfield 86FG9GJ9+ 39.380001 -89.480003 United States
Springfield 87JCH2R2+ 42.590000 -72.000000 United States
     Gondal 7JHGP32X+ 21.700001  70.099998         India
   Jamnagar 7JHGP32X+ 21.700001  70.099998         India
     Jetpur 7JHGP32X+ 21.700001  70.099998         India
   Junagadh 7JHGP32X+ 21.700001  70.099998         India
  Porbandar 7JHGP32X+ 21.700001  70.099998         India
     Rajkot 7JHGP32X+ 21.700001  70.099998         India
    Veraval 7JHGP32X+ 21.700001  70.099998         India


<br>

## 3. CSV Export

Beide DataFrames werden im aufgearbeiteten Zustand als CSVs gespeichert.
- clim_change_celsius_df.csv
- clim_change_geo_df.csv

In [36]:
""" Dauer etwa 23 Sekunden """

# Dataframes als CSV speichern --> ca. 23 Sekunden
celsius_df.to_csv('clim_change_celsius_df.csv', index=False)
geo_df.to_csv('clim_change_geo_df.csv', index=False)

<br>

## 4. Speichervergleich
Überarbeitete DataFrames im Vergleich zum DataFrame im Rohzustand mit relevanten Spalten

**Nach der Transformation konnte die Speichernutzung ingesamt um über 56 % reduziert werden**

- main_df 2.3 GB (Rohzustand)
- celsius_df 1004.6 MB (überarbeitet)
- geo_df 621.1 KB (überarbeitet)

Im Vergleich etwa **2.3 GB --> 1.0 GB**

In [37]:
""" Dauer etwa 19 Sekunden """

# Einlesen relevanter Spalten aus der Haupt-Datenquelle --> ca. 7 Sekunden
main_df = pd.read_csv(
    'GlobalLandTemperaturesByCity.csv',
    usecols=['dt', 'City', 'Country', 'AverageTemperature', 'Latitude', 'Longitude']
)

# DataFrames Speichernutzung im Vergleich --> ca. 12 Sekunden
main_df.info(memory_usage='deep')
print('')
celsius_df.info(memory_usage='deep')
print('')
geo_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 6 columns):
 #   Column              Dtype  
---  ------              -----  
 0   dt                  object 
 1   AverageTemperature  float64
 2   City                object 
 3   Country             object 
 4   Latitude            object 
 5   Longitude           object 
dtypes: float64(1), object(5)
memory usage: 2.3 GB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8235082 entries, 0 to 8235081
Data columns (total 4 columns):
 #   Column              Dtype         
---  ------              -----         
 0   dt                  datetime64[ns]
 1   AverageTemperature  float32       
 2   City                string        
 3   plus_code           object        
dtypes: datetime64[ns](1), float32(1), object(1), string(1)
memory usage: 1004.6 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510 entries, 0 to 3509
Data columns (total 5 columns):
 #   Column     Non-Null Coun